# Section 9: EM
Notes by Mary Richardson (2020)

## Expectation Maximization

We've discussed several applications of EM in this course so far:
1. gene expression clustering (soft k-means or mixture models)
2. RNA-seq read mapping
3. motif finding

EM is a very powerful tool for optimizing parameters when you are missing critical and dependent information. For example:
1. for gene expression clustering, we don't know the true centroids or the datapoints that should be assigned to each cluster
2. for read mapping, we don't know the expected read counts for each transcript or the transcript abundances
3. for motif finding, we don't know the expected letter counts for each motif position or the motif letter frequencies

EM let's us update these unknown quantities simultaneously. In the **expectation** step, we compute the probability distribution using current parameters and often assign "counts" of some sort:
1. calculate probabilities of each gene belonging to each cluster
2. calculate expected read counts for each transcript
3. calculate expected letter counts for each motif

In the **maximization** step, we update the parameters based on the results of the expectation step:
1. update centroid locations
2. update nucleotide abundances
3. update motif letter frequencies

This continues in a cycle until convergence, which by now is probably a familiar idea to you.

If you're interested in other applications of EM in computational biology (there are lots!), [this](https://www.nature.com/articles/nbt1406) article provides a helpful though not comprehensive review.

## Random variables

Let's start by summarizing what we know about the model we're going to build.

We **observe** the the read sequence:
- $R$ is the observed read sequence

We have **hidden** (or **latent**) variables that we don't directly observe, but that the model uses to generate the observed data:

- $G$ is the transcript isoform (ranges from $i = 1..M$)
- $S$ is the start position of the read (ranges from $s = 1..L_i$)

We want to **infer** the expression level of each transcript:
- $\theta$ represents the parameters, which are the _nucleotide_ abundances $\nu$

## Probabilistic graphical models

#### Now let's look at the probabilistic graphical model to understand the relationship between our random variables.

**Probabilistic graphical models** (aka Bayesian networks) are directed acyclic graphs (DAGs). In this problem set we know the underlying causal relationships between the variables, so we use a directed graph to represent our model. Probabilistic graphical models allow us to visualize the dependencies of random variables and see which variables are independent of the other variables. This helps us see the assumptions of our model (and as you saw in lecture, it helps us simplify the joint probability calculation!) The shaded variable is our **observed** data.

<img src="model.png" width=400 height=400 />

## Generative Model

#### First, a word of caution...

Expression levels are given in transcript abundance ($\tau$) in input and output, but we need them to be in nucleotide abundance ($\nu$) for our generative model. Remember that you can convert between them using the following equation:

$$\nu_i = \frac{\tau L_i}{\sum_i{\tau L_i}}$$

#### Let's use the model drawn above to simulate positive control data that we can later use to test our algorithm.

- **Step 1**: Select a transcript isoform $G=i$ with probability $\nu_i$


- **Step 2**: Select a start position $S=s$ with probability $\frac{1}{L_i-\ell}$ (assuming a uniform distribution)


- **Step 3**: Generate a read sequence $R_n$, given $G=i$ and $S=s$

In [ ]:
def simulate_reads(N, theta):
    '''
    Given N (the number of reads to sample),
    and theta, which includes nu (nucleotide abundance) and L (transcript lengths),
    Simulate reads
    '''

## Log Likelihood of Model

#### Let's start by writing the likelihood of a single read. 

We want to find the probability that the model generates the observed reads $R$, given the parameters of the model $\theta$:

$$P(R|\theta)$$

But what we actually have is:

$$P(R,S,G|\theta)$$

So we need to marginalize over the **hidden** variables in our problem ($S$ and $G$):

$$P(R|\theta) = \sum_S \sum_G P(R,S,G|\theta)$$

#### Next, let's rewrite this likelihood equation in terms of probabilities we can actually calulate.

We can expand our likelihood into a product of conditional probabilities:

$$P(R|\theta) = \sum_S\sum_G{P(R|S,G,\theta)P(S|G,\theta)P(G|\theta)}$$


Using the graphical model above, we can determine the following:

- The probability of selecting isoform $G$ is conditional on $\theta$:  
$$P(G|\theta)$$


- $S$ and $\theta$ are conditionally independent given $G$:  
$$P(S|G,\theta) = P(S|G)$$


- $R$ is conditionally independent of $\theta$ given $S$ and $G$:  
$$P(R|S,G,\theta) = P(R|S,G)$$

Using these independence assumptions, we can simplify our probability:

$$P(R|\theta) = \sum_S\sum_G{P(R|S,G)P(S|G)P(G|\theta)}$$

#### Now, let's write these conditional probabilities in terms of variables we know.

$P(R|S,G)$ is the probability of a read given the segment and gene. 
- We either observe the read sequence at the given position in the transcript 
$$P(R|S,G) = 1$$ 
- Or we don't
$$P(R|S,G) = 0$$

We can write this more generally:
$$P(R|S,G) =  \left\{
\begin{array}{ll}
      1 & \text{if } R[1..\ell] = G_i[s..s+\ell] \\
      0 & \text{otherwise} \\
\end{array} 
\right.$$

$P(S|G)$ is the probability that a segment $S$ is in the given transcript $G$. This should be inversely propositional to the length of that transcript:

$$P(S|G) = \frac{1}{L_i-\ell}$$

$P(G|\theta)$ is the probability of a transcript given the parameters, which is equal to the nucleotide abundance of that transcript:

$$P(G|\theta) = \nu_i$$

#### Finally, let's write the likelihood of a single read.

Now our likelihood calculation is:

$$P(R|\theta) = \sum_S\sum_G P(R,S,G|\theta) = \sum_S\sum_G P(R|S,G)P(S|G)P(G|\theta) = \sum_S\sum_G \left\{
\begin{array}{ll}
      \frac{\nu_i}{L_i-\ell} & \text{if } R[1..\ell] = G_i[s..s+\ell] \\
      0 & \text{otherwise} \\
\end{array} 
\right.$$

#### We can now find the total likelihood of all reads.

We assume each read is independent. Therefore, our total likelihood is the product of the likelihood of each individual read:
$$Likelihood = \prod_N{P(R_n|\theta})$$

So in log space:
$$LogLikelihood = \sum_N{logP(R_n|\theta})$$

In [ ]:
def log_likelihood(R, theta):
    '''
    Given R (reads) and 
    theta (nucleotide abundances and transcript lengths),
    Calculate the total log likelihood of the model
    '''

## Expectation Maximization

####  Since we have a "classic chicken and the egg problem," we can use EM to optimize our parameters.

We want to find which transcript each of the reads came from. If we knew the counts of reads mapping to each transcript, we could estimate the parameters $\theta$. If we knew $\theta$, we could estimate the counts. But we don't know either one. So we use EM! 

#### First, let's estimate the expected counts.

For the **expectation** step, we infer the expected counts $c_i$ mapping to each transcript, given the current parameters $\theta$. This requires calculating the probability that a read mapped to each transcript:
$$P(G|R,\theta) = \frac{\sum_S P(R,G,S|\theta)}{\sum_G \sum_S P(R,G,S|\theta)}$$

But we know from our likelihood calculation that $P(R,G,S|\theta)$ is, so:

$$P(G|R,\theta) = \frac{\sum_S \frac{\nu_i}{L_i-\ell} \text{ if } R[1..\ell] = G_i[s..s+\ell]} {\sum_S\sum_G \frac{\nu_i}{L_i-\ell} \text{ if } R[1..\ell] = G_i[s..s+\ell]}$$

Now to get the counts assigned to each transcript, we sum over the probabilities for all reads:

$$c_i = \sum_n P(G_n=i|R_n,\theta)$$

In [ ]:
def expectation(R, theta):
    '''
    Given R (reads) and 
    theta (nucleotide abundances and transcript lengths),
    Calculate expected counts
    '''

#### Next, let's update the nucleotide abundances.

For the **maximization** step, we calculate the updated $\theta$, given the $c_i$ from the previous expectation step. To do this, we simply normalize the counts in order to get nucleotide abundances:
$$\nu_i = c_i/\sum_j c_j = c_i/N$$

In [ ]:
def maximization(C):
    '''
    Given C (read counts) and 
    Caluclate the updated nu (nucleotide abundances)
    '''

#### Now we're ready to run EM! 

- **Step 1**: initialize $\theta$ to anything (can be random)


- **Step 2**: (expectation) calculate $c_i = \sum_n P(G_n=i|R_n,\theta)$


- **Step 3**: (maximization) calculate updated $\theta_i = c_i/N$


- **Step 4**: repeat EM until converged


EM finds a local optimum, *but* Li et al. (2010) show that there is only a single optimum for $\theta$ so we will be finding the global optimum in this case. This means we can initialize our parameters however we want as long as the nucleotide abundances sum to one. (**Hint:** check out the section in the lecture notes on the Dirichlet distribution.)

In [ ]:
def expectation_maximization(R, theta):
    '''
    Given R (reads) and 
    theta (nucleotide abundances and transcript lengths),
    Run EM until convergence
    '''

#### One final tip...
For this problem you should just operate with nucleotide abundances during the EM and then convert to transcript abundances after convergence.

In [ ]:
# Convert nucleotide abundance to transcript abundance
